In [1]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")

/home/cloudcraftz/anaconda3/envs/rag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

In [4]:
qdrant = QdrantVectorStore.from_existing_collection(
    embedding=embeddings,
    collection_name="demo_collection",
    path="./tmp/langchain_qdrant"
)

In [48]:
import requests
import json

In [59]:
def get_answer(question):
    
    results = qdrant.similarity_search(
    question, k=5)

    retrieved_info = ""
    for i, res in enumerate(results):
        retrieved_info += f"Chunk {i+1}:\n{res.page_content}\nSource: {res.metadata['source']}\nUrl: {res.metadata['url']}\n\n"

    prompt = f"""
                You are a helpful medical assistant. You are required to answer users questions with the help of provided chunks only. If the chunks does not contain  the answer, politely say that Sorry I dont know the answer and dont give any information. 
                Each chunk also has source and url. if you are answering from a chunk, do mention the source and url with the answer.
                Question : {question}
                Chunks" {retrieved_info}
            """
    
    # API request to Ollama
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": "gemma3:4b",
        "prompt": prompt,
        "stream": True
    }

    with requests.post(url, json=payload, stream=True) as r:
        for line in r.iter_lines():
            if line:
                data = json.loads(line)
                # Each chunk contains part of the response in 'response'
                print(data.get("response", ""), end="", flush=True)

    print()  # final newline
    
    

In [61]:
question = "what treatments are there for gonorrhoeae"
get_answer(question)

To treat gonorrhoeae, you’ll need antibiotics. Typically, this involves one shot (injection) of ceftriaxone. Your provider will determine your dosage based on your weight. If you’re allergic to ceftriaxone, your healthcare provider may give you a shot of gentamicin plus azithromycin, which is an antibiotic you take orally. (Source: CleavelandClinic, Url: https://my.clevelandclinic.org/health/diseases/4217-gonorrhea) 

Additionally, your provider may prescribe additional medication, like doxycycline, to clear a co-infection with chlamydia. (Source: CleavelandClinic, Url: https://my.clevelandclinic.org/health/diseases/4217-gonorrhea)
